# CS 195: Natural Language Processing
## Introduction to the Hugging Face Transformers Library


## References

Hugging Face *Quicktour*: https://huggingface.co/docs/transformers/quicktour

Hugging Face *Run Inference with Pipelines tutorial*: https://huggingface.co/docs/transformers/pipeline_tutorial

Hugging Face *NLP Course, Chapter 2*: https://huggingface.co/learn/nlp-course/chapter2/1

## Announcement: Venture Capital Panels

From Kevin Croft (Director of the Kelley Center for Insurance Innovation):

Do you know a student interested in innovation, starting a business, or with great tech solutions who would like to learn from those who’ve done it before?  The Kelley Center for Insurance Innovation invites you and your students to two opportunities to engage with business accelerators and venture capital investors.  Please share this information with your students!
 * On Wednesday, September 27 at 4 pm in Aliber 101 the leaders of the Global Insurance Accelerator and BrokerTech Ventures will form a panel to discuss how accelerators work, the application/acceptance process, the mentorship process, and success stories from their accelerators.   
 * On Wednesday, October 4 at 4 pm in Aliber 101 Wellabe Ventures, ManchesterStory, and Homesteaders Life will form a panel to discuss their approach to private equity investing in innovation partners.  The discussion will include identification/selection of target firms, the evaluation process, goals of the investment, and success stories.

I hope to see you there!

## The Plan

The first thing we're going to do is get comfortable with the Hugging Face *Transformers* library for Python

Eventual goals:
* understand and explain how transformer models work
* create and adapt transformers for a specific purpose

For now:
* learn to *use* existing transformer models
* understand *what* they can do
* get a feel for pupular families of models and how they're related

We will dig into the internals later

## What is Hugging Face?

Hugging Face is a private company
* Founded in 2016 by French entrepreneurs Clément Delangue, Julien Chaumond, and Thomas Wolf
* Based in New York City

Provide a popular free, open-source Python library called **transformers** for NLP (and other) tasks

Host *hundreds of thousands of models* that you can use in your own programs

## Installing the transformers module

This is my favored way of installing packages from a Jupyter Notebook

If you have lots of Python distributions installed, it should use the right one

It may take a few minutes, but *you should only have to do this once*

In [1]:
import sys
!{sys.executable} -m pip install transformers


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


## Using the sentiment analysis pipeline

**Sentiment analysis** attempts to identify the overall feeling intended by the writer of some text

The creators of this model **trained** it on lots of examples of text that were labeled as either *positive* or *negative*

A **pipeline** is a series of steps for performing **inference**
* tokenize and preprocess the input text (more on this later)
* ask the model for a prediction
* post-process model's result and turn it into something you can use

![full_nlp_pipeline.svg](https://github.com/ericmanley/f23-CS195NLP/blob/main/images/full_nlp_pipeline.svg?raw=1)
image source: https://huggingface.co/learn/nlp-course/chapter2/2?fw=pt

We *are* specifying the kind of task: `sentiment-analysis`

We *are not* asking for a specific model, so it picks one of many it has by default

The first time you do this, it will have to download the model - this can take some time depending on your network connection

In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

classifier("I love how easy it is to build sentiment-aware applications with the transformers library!")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


[{'label': 'POSITIVE', 'score': 0.9984305500984192}]

**Test it out:** Try changing the input to get different labels/scores

## Working with batches of text

To get classifications of many different examples, pass in a list of strings.

In [3]:
results = classifier(["It's really cool that you can get classifications for a whole batch of text",
                      "I wonder if the rest of the class will be this easy.",
                     "Spolier alert: it won't be."])
print(results)

[{'label': 'POSITIVE', 'score': 0.9991173148155212}, {'label': 'NEGATIVE', 'score': 0.9557347297668457}, {'label': 'NEGATIVE', 'score': 0.9962737560272217}]


Note that the results come back as a list of dictionaries, so you can manipulate it in the normal ways.

In [4]:
print("The sentence had",results[0]["label"],"sentiment, with a score of",results[0]["score"])

The sentence had POSITIVE sentiment, with a score of 0.9991173148155212


## Exercise: Specifying a model

Now try asking for a specific model.

Replace one line of code in your earlier example.

In [5]:
classifier = pipeline("sentiment-analysis", model="SamLowe/roberta-base-go_emotions")

ValueError: Could not load model SamLowe/roberta-base-go_emotions with any of the following classes: (<class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForSequenceClassification'>, <class 'transformers.models.roberta.modeling_tf_roberta.TFRobertaForSequenceClassification'>).

How is this model different from the first model?

Create a cell in this notebook and note the differences you see

In [ ]:
text_list= [
    "Dogs are mammals.",
    "It's really cool that you can get classifications for a whole batch of text",
    "I wonder if the rest of the class will be this easy.",
    "Spolier alert: it won't be.",
    "Today is Thursday.",
    ]
results = classifier(text_list)
print(results)

[{'label': 'neutral', 'score': 0.9344079494476318}, {'label': 'admiration', 'score': 0.5409085750579834}, {'label': 'surprise', 'score': 0.7312608957290649}, {'label': 'neutral', 'score': 0.7667409181594849}, {'label': 'neutral', 'score': 0.5689737200737}]


In [ ]:
for i in range(len(text_list)):
  print(f"'{text_list[i]}' was labeled {results[i]['label']} with a score of {results[i]['score']}")

'Dogs are mammals.' was labeled neutral with a score of 0.9344079494476318
'It's really cool that you can get classifications for a whole batch of text' was labeled admiration with a score of 0.5409085750579834
'I wonder if the rest of the class will be this easy.' was labeled surprise with a score of 0.7312608957290649
'Spolier alert: it won't be.' was labeled neutral with a score of 0.7667409181594849
'Today is Thursday.' was labeled neutral with a score of 0.5689737200737


In [ ]:
sentences = [
    "The quarterly financial report is showing an improvements this quarter.",  # Incorrect
    "Him has completed the audit for last fiscal year.",  # Incorrect
    "Please to inform the board about the recent developments.",  # Incorrect
    "The team successfully achieved all its targets for the last quarter.",  # Correct
    "Our company is exploring new ventures in the European market."  # Correct
]


In [ ]:
results = pipe(sentences)
for i in range(len(results)):

  print(results[i])

{'label': 'LABEL_0', 'score': 0.5484333038330078}
{'label': 'LABEL_0', 'score': 0.6328231692314148}
{'label': 'LABEL_0', 'score': 0.542167067527771}
{'label': 'LABEL_0', 'score': 0.7175154685974121}
{'label': 'LABEL_0', 'score': 0.6996312141418457}


There are different/more labels

## Applied Exploration

The `roberta-base-go_emotions` model is documented here: https://huggingface.co/SamLowe/roberta-base-go_emotions

Answer some questions about this:
* What is `roberta-base`? Write down some things you can learn about it from the documentation.
* What is `go_emotions`? Write down some things you can learn about it from the documentation.

Go to the Hugging Face models page: https://huggingface.co/models
* click `Text Classification`
* Try some additional models
    - test out at least one more sentiment/emotions model
    - test out at least two other kinds of models - like news topic classification or spam detection
    - write down some info about the models you found
        - what is it for?
        - who made it?
        - what kind of data was it trained on?
        - are they based on some other model and trained on new data (*fine-tuned*) for a specific task?